In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import Gemma3ForConditionalGeneration
model_id = "google/gemma-3-4b-pt"

model = Gemma3ForConditionalGeneration.from_pretrained(model_id)



print(model)


config.json:   0%|          | 0.00/815 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Gemma3ForConditionalGeneration(
  (model): Gemma3Model(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(4096, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwi

In [4]:
siglip = model.vision_tower
print(siglip)




SiglipVisionModel(
  (vision_model): SiglipVisionTransformer(
    (embeddings): SiglipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
      (position_embedding): Embedding(4096, 1152)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-26): 27 x SiglipEncoderLayer(
          (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
          (self_attn): SiglipAttention(
            (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
            (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
            (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
            (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
          )
          (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=1152, out_f

In [5]:
for name, module in siglip.named_modules():
    print(name, type(module))

 <class 'transformers.models.siglip.modeling_siglip.SiglipVisionModel'>
vision_model <class 'transformers.models.siglip.modeling_siglip.SiglipVisionTransformer'>
vision_model.embeddings <class 'transformers.models.siglip.modeling_siglip.SiglipVisionEmbeddings'>
vision_model.embeddings.patch_embedding <class 'torch.nn.modules.conv.Conv2d'>
vision_model.embeddings.position_embedding <class 'torch.nn.modules.sparse.Embedding'>
vision_model.encoder <class 'transformers.models.siglip.modeling_siglip.SiglipEncoder'>
vision_model.encoder.layers <class 'torch.nn.modules.container.ModuleList'>
vision_model.encoder.layers.0 <class 'transformers.models.siglip.modeling_siglip.SiglipEncoderLayer'>
vision_model.encoder.layers.0.layer_norm1 <class 'torch.nn.modules.normalization.LayerNorm'>
vision_model.encoder.layers.0.self_attn <class 'transformers.models.siglip.modeling_siglip.SiglipAttention'>
vision_model.encoder.layers.0.self_attn.k_proj <class 'torch.nn.modules.linear.Linear'>
vision_model.enc

In [6]:
!pip install torchinfo

In [7]:
from torchinfo import summary

summary(siglip, input_size=(1, 3, 896, 896), depth=3)

Layer (type:depth-idx)                                  Output Shape              Param #
SiglipVisionModel                                       [1, 4096, 1152]           --
├─SiglipVisionTransformer: 1-1                          [1, 4096, 1152]           --
│    └─SiglipVisionEmbeddings: 2-1                      [1, 4096, 1152]           --
│    │    └─Conv2d: 3-1                                 [1, 1152, 64, 64]         678,528
│    │    └─Embedding: 3-2                              [1, 4096, 1152]           4,718,592
│    └─SiglipEncoder: 2-2                               [1, 4096, 1152]           --
│    │    └─ModuleList: 3-3                             --                        411,466,608
│    └─LayerNorm: 2-3                                   [1, 4096, 1152]           2,304
Total params: 416,866,032
Trainable params: 416,866,032
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 3.20
Input size (MB): 9.63
Forward/backward pass size (MB): 11055.66
Params size (MB): 1667

In [8]:
siglip.save_pretrained("./siglip_from_gemma3-4b")


In [ ]:
import shutil
shutil.make_archive("siglip_from_gemma3-4b", 'zip', "./siglip_from_gemma3-4b")


In [ ]:
from google.colab import files
files.download("siglip_from_gemma3-4b.zip")


In [10]:
from huggingface_hub import HfApi, create_repo
from transformers import SiglipVisionModel

# Choose your repo name
repo_name = "siglip-from-gemma3-4b"
create_repo(repo_id=repo_name, private=False)

# Push model
siglip.push_to_hub(repo_name)


model.safetensors:   0%|          | 0.00/1.67G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DrishtiSharma/siglip-from-gemma3-4b/commit/42fe0f4fc9c4504b7c5c5b90ef95e5b2895e40c7', commit_message='Upload model', commit_description='', oid='42fe0f4fc9c4504b7c5c5b90ef95e5b2895e40c7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DrishtiSharma/siglip-from-gemma3-4b', endpoint='https://huggingface.co', repo_type='model', repo_id='DrishtiSharma/siglip-from-gemma3-4b'), pr_revision=None, pr_num=None)

In [11]:
import transformers
print(transformers.__version__)


4.53.1
